In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import math
import pickle
import numpy as np
import pandas as pd
import argparse
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
# from functions import *
# from model import PredOT
import warnings
import random
import esm

/opt/anaconda3/envs/myCDC/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
        device = torch.device('cuda')
        print('We use CUDA!')
else:
    device = torch.device('cpu')
    print('We use CPU!')
    
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model = model.to(device)
model.eval() 

We use CUDA!


ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bia

In [ ]:
split='train'
ca_path="/usr/data/protein/go"

fasta_file = os.path.join(ca_path, split+'.fasta')


# Load the fasta file.
protein_seqs = []
with open(fasta_file, 'r') as f:
    protein_name = ''
    for line in f:
        if line.startswith('>'):
            protein_name = line.rstrip()[1:]
        else:
            amino_chain = line.rstrip()
            protein_seq = [(protein_name, amino_chain)]
            batch_labels, batch_strs, batch_tokens = batch_converter(protein_seq)
            batch_tokens = batch_tokens.to(device)
            with torch.no_grad():
                emb = model(batch_tokens, repr_layers=[33], return_contacts=False)
                emb = emb["representations"][33]
            emb = emb.cpu().numpy()
            print(emb.shape)
            np.save('/usr/data/myCDConv/data/go/esm2_emb/{}.npy'.format(protein_name), emb)